# Programming with Python

## Lecture 11: Concurrency 3

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 03 May, 2025

# Multi-processing

## `multiprocessing` module

### Overview

The `multiprocessing` module provides a way to create new processes using an API similar to the `threading` module. Unlike threads, it uses separate subprocesses, which allows it to bypass the Global Interpreter Lock (GIL) and take full advantage of multiple CPU cores. This enables true parallel execution and works on both POSIX systems and Windows.

In addition to thread-like functionality, `multiprocessing` includes features not found in the `threading` module. One notable feature is the `Pool` class, which simplifies running a function in parallel across a collection of inputs—this is known as data parallelism.

`multiprocessing.Process` objects represent activity that is run in a separate process. The `multiprocessing.Process` class has equivalents of all the methods of `threading.Thread`.

The `if __name__ == '__main__'` part is necessary in multi-processing as you can see in the following example. This is to make sure that the main module can be safely imported by a new Python interpreter without causing unintended side effects (such as starting a new process).

**See practical example 1**.

### Contexts and start methods

#### 1. **`spawn`**
- **How it works:** Starts a *fresh Python interpreter process*.
- **Pros:** Clean slate—only essential resources are inherited.
- **Cons:** Slower startup.
- **Default on:** **Windows** and **macOS**.

#### 2. **`fork`**
- **How it works:** Uses `os.fork()`. Child is a clone of the parent.
- **Pros:** Very fast.
- **Cons:** Not safe with multithreaded processes.
- **Default on:** Most **Linux**/POSIX systems (but **changing in Python 3.14**).
- **Deprecated** for multi-threaded environments since Python 3.12 because forking a multi-threaded process is problematic.

#### 3. **`forkserver`**
- **How it works:** Starts a **server** process which handles forking. As it is single-threaded, it is safer to use `fork` method.
- **Pros:** Safer than `fork`, faster than `spawn`. No excess resources inherited.
- **Cons:** Requires OS support for file descriptor passing.
- **Available on:** POSIX with certain features (e.g., Linux).

Here are two ways to select a start method.

#### Option 1: `set_start_method()` (set once per program)

```python
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    mp.set_start_method('spawn')  # 'spawn', 'fork', or 'forkserver'
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()
```

#### Option 2: `get_context()` (preferred for libraries or multiple modes)

This avoids conflicts with other parts of the app or external libraries.

```python
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    ctx = mp.get_context('spawn')
    q = ctx.Queue()
    p = ctx.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()
```

## Inter-Process Communication (IPC)

Inter-Process Communication (IPC) refers to mechanisms that allow processes to exchange data and coordinate their actions. These mechanisms are essential for building complex systems where multiple processes need to work together. Synchronization, shared memory, queues and pipes are some examples for organizing IPC.

### Synchronization between processes

The `multiprocessing` module offers the same synchronization tools as the `threading` module. For example, the following example shows a race condition by incrementing a shared counter from multiple processes without a lock.

**See practical example 2**.

We can use **lock** to safely update the shared counter value and prevent race conditions.

**See practical example 3**.

# Exchanging objects between processes

When using multiple processes, one generally uses message passing for communication between processes and avoids having to use any synchronization primitives like locks.

For passing messages one can use `multiprocessing.Pipe()` (for a connection between two processes) or a queue (which allows multiple producers and consumers).

### `multiprocessing.Pipe([duplex])`

Returns a pair `(conn1, conn2)` of `multiprocessing.connection.Connection` objects representing the ends of a pipe.

If `duplex` is `True` (the default) then the pipe is bidirectional. If `duplex` is `False` then the pipe is unidirectional: `conn1` can only be used for receiving messages and` conn2` can only be used for sending messages.

Note that data in a pipe may become corrupted if two processes (or threads) try to read from or write to the same end of the pipe at the same time. Of course there is no risk of corruption from processes using different ends of the pipe at the same time.

The `send()` method serializes the object using `pickle` and the `recv()` re-creates the object.

#### Key Characteristics

- **Byte stream:** Pipes typically handle an unstructured stream of bytes. The writing process sends bytes, and the reading process receives bytes, often without inherent message boundaries. The reader needs to know how to interpret the byte stream (e.g., reading until a newline character or reading a fixed number of bytes).
- **Kernel-managed buffer:** The operating system manages a buffer for the pipe. If the writer produces data faster than the reader consumes it, the data accumulates in the buffer. If the buffer fills up, the writer will block (wait) until the reader consumes some data. Conversely, if the reader tries to read from an empty pipe, it will block until the writer sends data.
- **Synchronization:** The blocking behaviour provides implicit synchronization between the producer (writer) and consumer (reader).

**See practical example 4**.

### Queues

- Similar to pipes, these are another mechanism for Inter-Process Communication (IPC).
- **Message-Oriented:** Unlike pipes which handle byte streams, message queues typically handle discrete messages. The sender enqueues a whole message, and the receiver dequeues a whole message. This preserves message boundaries.
- **Many-to-Many:** Often, multiple processes can write to the same queue, and multiple processes can read from it (though often a message is consumed by only one reader).

In Python, the `multiprocessing.Queue`, `multiprocessing.SimpleQueue` and `multiprocessing.JoinableQueue` types are multi-producer, multi-consumer FIFO queues modelled on the `queue.Queue` class in the standard library. They differ in that `multiprocessing.Queue` lacks the `task_done()` and `join()` methods introduced into Python 2.5’s `queue.Queue` class.

If you use `JoinableQueue` then you must call `JoinableQueue.task_done()` for each task removed from the queue or else the semaphore used to count the number of unfinished tasks may eventually overflow, raising an exception.

One difference from other Python queue implementations, is that `multiprocessing` queues serializes all objects that are put into them using `pickle`. The object return by the `get` method is a re-created object that does not share memory with the original object.

Multi-processing queues are thread and process safe.

**See practical example 5**.

## Sharing state between processes

When doing concurrent programming it is usually best to avoid using shared state as far as possible. This is particularly true when using multiple processes.

However, `multiprocessing` provides a couple of ways of doing so, namely shared memory and server process.

### Shared memory

In Python multiprocessing, shared memory allows multiple processes to access and modify the same data without copying it between processes — improving performance and coordination.

Normally, when a new process is spawned, it gets its own copy of data (due to process isolation). Shared memory avoids this by letting processes point to the same data block.

In Python, this can be done with shared `ctypes` objects.

[`ctypes`](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

#### `multiprocessing.Value(typecode_or_type, *args, lock=True)`

Creates a shared object in memory, typically wrapped with a synchronization mechanism. This object is used to safely share simple data types (like an `int` or `float`) between processes.

- **`typecode_or_type`** specifies the data type, using either a ctypes type or a one-letter typecode (like `'i'` for integer).
- **`*args`** are passed to the constructor of the specified type.
- **`lock`** controls access:
  - `True` (default): uses an internal recursive lock for thread-safe access.
  - `False`: disables synchronization (not safe for concurrent writes).
  - You can also pass your own `Lock` or `RLock`.

To modify the value safely in concurrent settings, **wrap the operation in a lock**:

```python
with counter.get_lock():
    counter.value += 1
```

This is necessary because operations like `+=` are **not atomic** — they involve both reading and writing the value.

####  `multiprocessing.Array(typecode_or_type, size_or_initializer, *, lock=True)`

Creates a **shared array** in memory for use across multiple processes, with optional synchronization.

- **`typecode_or_type`** defines the element type, using either a ctypes type or a one-character typecode (like `'i'` for integers).
- **`size_or_initializer`** can be:
  - An **integer**: creates a zero-initialized array of that length.
  - A **sequence**: initializes the array with the given values, and the sequence’s length sets the array size.
- **`lock`** controls concurrent access:
  - `True` (default): uses an internal lock for safe access.
  - `False`: no locking — not safe for simultaneous writes.
  - You can also pass a custom `Lock` or `RLock`.

The returned object is a synchronized wrapper unless `lock=False`.

---

The following shows common type codes:

| Type      | Typecode | Description              |
|-----------|----------|--------------------------|
| `int`     | `'i'`    | Signed integer (4 bytes) |
| `double`  | `'d'`    | Double-precision float   |
| `float`   | `'f'`    | Single-precision float   |
| `char`    | `'c'`    | Char (1 byte, `bytes`)   |
| `byte`    | `'b'`    | Signed char (-128 to 127)|

**See practical example 6**.

#### `multiprocessing.sharedctypes` module

The `multiprocessing.sharedctypes` module offers more flexibility than `Value` and `Array` by allowing you to create arbitrary `ctypes` structures in shared memory.

It allows you to:

- Define custom `ctypes` structures, arrays, and types
- Allocate them in shared memory so multiple processes can access them
- Use familiar `ctypes` declarations (e.g. `c_int`, `c_double`, `Structure`, etc.)

**See practical example 7**.

### Server process

#### `multiprocessing.Manager()`

In short, server process via `multiprocessing.Manager()` has the following properties:

- Creates proxy objects (e.g. `list`, `dict`) for sharing more complex data.
- Slower than `Value`/`Array` — works via a server process and proxy objects, not true shared memory.
- Easier for things like shared dictionaries, nested structures.

An object created by `multiprocessing.Manager()` runs a **server process** that hosts Python objects, allowing multiple processes to interact with those objects through **proxies**. Managers provide a way to create data which can be shared between different processes, including sharing over a network between processes running on different machines. A manager object controls a server process which manages shared objects. Other processes can access the shared objects by using proxies.

A **proxy** is an object that acts as a **remote reference** to a shared object managed by a `Manager`.

Instead of giving each process direct access to the actual object (which might live in a different memory space), the manager gives them a **proxy object**. This proxy communicates with the manager's **server process** under the hood to:
- Get or set data
- Call methods
- Synchronize access

The manager can be used to create and manage shared versions of common data types such as `list`, `dict`, `Namespace`, and threading synchronization primitives like `Lock`, `RLock`, `Semaphore`, `Condition`, `Event`, `Barrier`, as well as shared `Queue`, `Value`, and `Array` objects.

**See practical example 8**.

#### Customized managers

**Customized managers** allow you to extend or register your own types to be shared between processes via proxies, beyond the built-in ones like `list`, `dict`, etc.

A customized manager is created by:
1. Subclassing `multiprocessing.managers.BaseManager`.
2. Registering custom classes or callables via `register()` classmethod.
3. Starting the manager to allow processes to access the shared objects via proxies.

**See practical example 9**.

#### Remote managers

**Remote managers** allows you to share Python objects across machines or over a network, not just between processes on the same system. This is a powerful way to build distributed systems using the same proxy model.

A remote manager is an instance of `BaseManager` that:
- Runs a server process on a specific `host:port`
- Exposes shared objects to other Python processes, even on different machines
- Clients connect to this manager and use proxies to interact with the shared objects


##### Server code

**See practical example 10.1**.

##### Clients code

**See practical example 10.2 and 10.3**.

## A pool of workers

The `multiprocessing.Pool` class allows you to manage a **group of worker processes** that can handle tasks concurrently. You can submit jobs to this pool, and it will distribute them among the available workers.

- The pool supports **parallel map operations** and can handle **asynchronous jobs**, including **timeouts** and **callbacks**.
- Only the **process that creates the pool** should call its methods.

Here's a short description of some key functions in the `multiprocessing.Pool`:

**Pool methods:**

- **`pool.map(function, iterable)`**: Applies a function to each item in an iterable in parallel and returns results in the original order. Blocks until all tasks complete.

- **`pool.imap_unordered(function, iterable)`**: Similar to map, but lazier and returns results as soon as they're ready, regardless of input order. Can be faster when processing times vary.

- **`pool.apply(function, args)`**: Applies a function with the given arguments. Runs in only one process and blocks until completion. Rarely used due to its blocking nature.

- **`pool.apply_async(function, args)`**: Non-blocking version of apply. Returns a result object immediately while computation happens in the background. Use `result.get()` to retrieve the actual result when needed.

**AsyncResult methods:**

- **`result.get(timeout=None)`**: Retrieves the result of an async operation. If `timeout` is specified and the operation takes longer, raises `TimeoutError`.

- **`result.ready()`**: Returns `True` if the call has completed.

- **`result.successful()`**: Returns `True` if the call completed without raising an exception. Will raise `ValueError` if the result is not ready.

**See practical example 11**.

## CPU-bound task

We already know that in Python CPU-intensive tasks are best handled with `multiprocessing` rather than `threading`, mainly due to the limitations of the Global Interpreter Lock (GIL).

The GIL ensures that only one thread executes Python bytecode at a time, even on multi-core processors. This means that:

- Threading does not provide real parallelism for CPU-bound tasks.
- Threads still take turns using the CPU, resulting in limited performance gain or even overhead from context switching.
- In contrast, multiprocessing creates separate processes, each with its own Python interpreter and memory space, allowing for true parallel execution across multiple CPU cores.

Key points

- Multiprocessing bypasses the GIL, enabling full CPU core usage.
- Threading is limited by the GIL for CPU-bound work.
- Multiprocessing is ideal for tasks like number crunching, image processing, or simulations.
- Threading is better suited for I/O-bound tasks (e.g., file reads, network requests).

In summary, due to the GIL, `threading` is ineffective for CPU-heavy workloads, whereas `multiprocessing` provides actual parallelism and improved performance.

Let's see this in action with one more example.

### Prime number checking

**See practical example 12**.

## References

- [multiprocessing — Process-based parallelism](https://docs.python.org/3/library/multiprocessing.html)